In [1]:
!git clone https://github.com/Nazar1997/Sparse_vector.git sparse_vector

Cloning into 'sparse_vector'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 81 (delta 37), reused 62 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 17.54 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [1]:
import pandas as pd
from tqdm import tqdm_notebook, tqdm
from collections import defaultdict
import numpy as np
from sparse_vector.sparse_vector import SparseVector
import os
from joblib import load, dump, Parallel, delayed

In [2]:
cell_type = "stem_cells"
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y','M']]

def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'./data_hg_38/dna/') if f"{chrom}_" in i])
    return ''.join([load(f"./data_hg_38/dna/{file}") for file in files])

In [3]:
lens_of_chroms = {chrom: len(chrom_reader(chrom)) for chrom in tqdm(chroms, desc="Loading Chromosome Lengths")}

Loading Chromosome Lengths: 100%|██████████| 25/25 [00:03<00:00,  7.27it/s]


In [4]:
def delete_empty_files(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        if os.path.isfile(file_path) and os.stat(file_path).st_size == 0:
            os.remove(file_path)
            print(f"Deleted empty file: {file_path}")

In [5]:
directory_path = f'./data_hg_38/features/raw/{cell_type}'
delete_empty_files(directory_path)

In [4]:
def sparser(input_path, output_path, lens_of_chroms, chroms, max_value=1000):
    loc_dd = {}

    df = pd.read_csv(input_path, header=None, sep='\t')
    df[0] = df[0].str.split('_').str[0]

    tqdm.get_lock().locks = []
    for chrom, sub_df in tqdm(df.groupby(0), desc=input_path):
        if chrom not in chroms:
            continue

        vec = np.zeros(lens_of_chroms[chrom])
        
        for inter in sub_df.values:
            if "g4" in input_path:
                value = int(inter[3][-1])
                if value > 3:
                    value = 1
                else:
                    value = 0
            elif "zdna" in input_path:
                value = 1
            else:
                value = inter[4]
                
            if value > 0:
                vec[inter[1]:inter[2]+1] = np.maximum(vec[inter[1]:inter[2]+1], value)
            
        np.clip(vec, None, max_value, out=vec) 

        loc_dd[chrom] = SparseVector(vec)
        
        dump(loc_dd, output_path, 3)   

# Max of (1000, value) for omics

In [4]:
input_path = f"./data_hg_38/features/raw/{cell_type}"
output_path = f"./data_hg_38/features/sparse/{cell_type}"

files = sorted(os.listdir(input_path)[::-1])
done_files = set([i[:-4] for i in os.listdir(output_path)])
files = [file for file in files if file[:-4] not in done_files]

In [8]:
Parallel(n_jobs = -1, 
         backend= "multiprocessing")(delayed(sparser)(f"{input_path}/{file}", f"{output_path}/{file[:-4]}.pkl", lens_of_chroms, chroms) 
                                      for file in files if file.endswith('.bed'))

./data_hg_38/features/raw/stem_cells/TFs_and_others_BRD2.bed: 100%|██████████| 15/15 [00:32<00:00,  2.19s/it]t]
./data_hg_38/features/raw/stem_cells/TFs_and_others_BCL11A.bed: 100%|██████████| 23/23 [00:47<00:00,  2.05s/it]
./data_hg_38/features/raw/stem_cells/TFs_and_others_BRD1.bed: 100%|██████████| 22/22 [00:47<00:00,  2.15s/it]t]
./data_hg_38/features/raw/stem_cells/TFs_and_others_BRD3.bed: 100%|██████████| 23/23 [00:47<00:00,  2.08s/it]]
./data_hg_38/features/raw/stem_cells/TFs_and_others_ASH2L.bed: 100%|██████████| 24/24 [00:52<00:00,  2.17s/it]]
./data_hg_38/features/raw/stem_cells/TFs_and_others_ATF4.bed: 100%|██████████| 25/25 [00:52<00:00,  2.12s/it]]]
./data_hg_38/features/raw/stem_cells/TFs_and_others_BACH1.bed: 100%|██████████| 25/25 [00:53<00:00,  1.74s/it]]
./data_hg_38/features/raw/stem_cells/TFs_and_others_BACH1.bed: 100%|██████████| 25/25 [00:53<00:00,  2.12s/it]
./data_hg_38/features/raw/stem_cells/TFs_and_others_ASCL1.bed:  96%|█████████▌| 24/25 [00:53<00:01,  2.00s

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [10]:
input_path="./data_hg_38/targets/raw"
output_path="./data_hg_38/targets/sparse"

sparser(f"{input_path}/zdna_Kouzine_hg38.bed", f"{output_path}/zdna.pkl", lens_of_chroms, chroms)
sparser(f"{input_path}/g4_EQ_hg38_lifted.bed", f"{output_path}/g4.pkl", lens_of_chroms, chroms)

./data_hg_38/targets/raw/g4_EQ_hg38_lifted.bed: 100%|██████████| 25/25 [04:07<00:00,  9.89s/it]


# Preserving real omics value

In [7]:
def count_max(input_path, lens_of_chroms, chroms):
    loc_dd = {}

    df = pd.read_csv(input_path, header=None, sep='\t')
    df[0] = df[0].str.split('_').str[0]

    mx = 0
    for chrom, sub_df in df.groupby(0):
        if chrom not in chroms:
            continue

        chrom_max = sub_df[4].max()
        if chrom_max > mx:
            mx = chrom_max
    
    return mx

In [8]:
input_path = f"./data_hg_38/features/raw/{cell_type}"
files = sorted(os.listdir(input_path)[::-1])

mx = 0
for file in tqdm(files):
    if file.endswith('.bed'):
        mx = max(mx, count_max(f"{input_path}/{file}", lens_of_chroms, chroms))
    
print(mx)

100%|██████████| 323/323 [00:41<00:00,  7.80it/s]

5382


In [9]:
input_path = f"./data_hg_38/features/raw/{cell_type}"
output_path = f"./data_hg_38/features/sparse_max_scaled/{cell_type}"

files = sorted(os.listdir(input_path)[::-1])
done_files = set([i[:-4] for i in os.listdir(output_path)])
files = [file for file in files if file[:-4] not in done_files]

In [10]:
Parallel(n_jobs = -1, 
         backend= "multiprocessing")(delayed(sparser)(f"{input_path}/{file}", f"{output_path}/{file[:-4]}.pkl", lens_of_chroms, chroms, mx) 
                                      for file in files if file.endswith('.bed'))

./data_hg_38/features/raw/stem_cells/Histone_H2Aub.bed: 100%|██████████| 1/1 [00:03<00:00,  3.10s/it]
./data_hg_38/features/raw/stem_cells/Histone_H2B.bed: 100%|██████████| 13/13 [00:22<00:00,  1.74s/it]it]it]t]
./data_hg_38/features/raw/stem_cells/Histone_H2AK120Ub.bed: 100%|██████████| 13/13 [00:24<00:00,  1.89s/it]t]
./data_hg_38/features/raw/stem_cells/Histone_H3.bed: 100%|██████████| 20/20 [00:37<00:00,  1.86s/it]3s/it]it]
./data_hg_38/features/raw/stem_cells/Histone_H3K14me3.bed: 100%|██████████| 21/21 [00:38<00:00,  1.81s/it]it]
./data_hg_38/features/raw/stem_cells/Histone_H2A.XS139ph.bed: 100%|██████████| 23/23 [00:43<00:00,  1.89s/it]
./data_hg_38/features/raw/stem_cells/Histone_H2A.Z.bed: 100%|██████████| 23/23 [00:43<00:00,  1.90s/it]it]
./data_hg_38/features/raw/stem_cells/Histone_H3.3.bed: 100%|██████████| 23/23 [00:43<00:00,  1.91s/it]it]
./data_hg_38/features/raw/stem_cells/Histone_H1B.bed: 100%|██████████| 24/24 [00:44<00:00,  1.85s/it].54s/it]
./data_hg_38/features/raw

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,